# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [1]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: `None`

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [2]:
def oracle010(qq):
    # YOUR CODE HERE
    # Apply X gates to the first and third qubits
    yield cirq.X(qq[0])
    yield cirq.X(qq[2])
    
    # Apply a Toffoli (CCZ) gate to the second qubit controlled by the first and third qubits
    yield cirq.CCX(qq[0], qq[2], qq[1])
    
    # Revert the first and third qubits to their original state
    yield cirq.X(qq[0])
    yield cirq.X(qq[2])

In [3]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───────X───────
          │
2: ───X───@───X───

In [4]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [5]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [6]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'101': 17, '110': 15, '100': 14, '011': 13, '010': 13, '001': 12, '111': 8, '000': 8})


13

In [7]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Graph $G$ has 5 vertices and 6 edges: (0,3), (0,4), (1,3), (1,4), (2,3), (2,4).

Write an oracle for the graph $G$ to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla. 

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.

In [44]:
def oracle2(qq):
    # YOUR CODE HERE
    # Initialize the list of edges
    edges = [(0, 3), (0, 4), (1, 3), (1, 4), (2, 3), (2, 4)]

    # Loop over each edge and check the vertices coloring
    for i, (u, v) in enumerate(edges):
        edge_qubit = 5 + i
        # If both u and v are the same color (both 0 or both 1), flag it using edge_qubit
        yield cirq.Moment(
            [
                cirq.CCNOT(qq[u], qq[v], qq[edge_qubit]),
            ]
        )
        yield cirq.Moment(
            [
                cirq.CNOT(qq[edge_qubit], qq[11]),
            ]
        )
        yield cirq.Moment(
            [
                cirq.CCNOT(qq[u], qq[v], qq[edge_qubit])
            ]
        )

In [45]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [46]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [47]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'00000': 44,
         '11001': 42,
         '11111': 40,
         '01110': 39,
         '00001': 39,
         '01010': 39,
         '10110': 39,
         '01111': 37,
         '00101': 36,
         '10001': 36,
         '11101': 35,
         '01011': 35,
         '10111': 34,
         '00111': 34,
         '00110': 33,
         '01101': 33,
         '01001': 32,
         '11110': 32,
         '00010': 32,
         '10100': 30,
         '01000': 29,
         '10011': 26,
         '00011': 25,
         '10000': 25,
         '11000': 24,
         '00100': 24,
         '11010': 24,
         '11011': 22,
         '11100': 21,
         '10101': 21,
         '01100': 20,
         '10010': 18})

In [48]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Graph $G$ has 4 vertices and 5 edges: (0,1), (0,2), (0,3), (1,2), (1,3)

Write an oracle for the graph $G$ to check whether there exists a coloring with at least 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 
- 0-3 for the vertices,
- 4-8 for the edges,
- 9-11 for the addition (remember we need three qubits here for addition unlike the last question), and
- 12 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.

In [61]:
def oracle3(qq):
    # YOUR CODE HERE
    # Initialize the list of edges
    edges = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3)]

    # Loop over each edge and check the vertices coloring
    for i, (u, v) in enumerate(edges):
        edge_qubit = 4 + i
        # If u and v are different colors, set edge_qubit
        yield cirq.Moment([cirq.CNOT(qq[u], qq[edge_qubit])])
        yield cirq.Moment([cirq.CNOT(qq[v], qq[edge_qubit])])
    
    # Sum the edge qubits to count how many edges have different colors
    yield cirq.Moment([cirq.CNOT(qq[4], qq[9])])
    yield cirq.Moment([cirq.CNOT(qq[5], qq[10])])
    yield cirq.Moment([cirq.CNOT(qq[6], qq[10])])
    yield cirq.Moment([cirq.CNOT(qq[7], qq[11])])
    yield cirq.Moment([cirq.CNOT(qq[8], qq[11])])
    
    # Calculate the sum of edge indicators into qubits 9-11
    # Assuming 9, 10, 11 represent a 3-bit sum
    yield cirq.Moment([cirq.CCNOT(qq[10], qq[11], qq[12])])
    yield cirq.Moment([cirq.CNOT(qq[9], qq[12])])
    
    # Check if the sum >= 4 by ensuring the sum qubits (9-11) are 100 or greater
    yield cirq.Moment([cirq.X(qq[10])])
    yield cirq.Moment([cirq.X(qq[11])])
    yield cirq.Moment([cirq.CCNOT(qq[9], qq[10], qq[12])])
    yield cirq.Moment([cirq.X(qq[10])])
    yield cirq.Moment([cirq.X(qq[11])])
    
    # Reset the edge qubits and sum qubits
    for i, (u, v) in enumerate(edges):
        edge_qubit = 4 + i
        yield cirq.Moment([cirq.CNOT(qq[v], qq[edge_qubit])])
        yield cirq.Moment([cirq.CNOT(qq[u], qq[edge_qubit])])
    
    yield cirq.Moment([cirq.CNOT(qq[4], qq[9])])
    yield cirq.Moment([cirq.CNOT(qq[5], qq[10])])
    yield cirq.Moment([cirq.CNOT(qq[6], qq[10])])
    yield cirq.Moment([cirq.CNOT(qq[7], qq[11])])
    yield cirq.Moment([cirq.CNOT(qq[8], qq[11])])
    
    yield cirq.Moment([cirq.CCNOT(qq[10], qq[11], qq[12])])
    yield cirq.Moment([cirq.CNOT(qq[9], qq[12])])
    yield cirq.Moment([cirq.CCNOT(qq[10], qq[11], qq[12])])

In [62]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))  

In [63]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [64]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'1011': 75,
         '1100': 72,
         '0101': 71,
         '1000': 69,
         '0011': 68,
         '1101': 67,
         '1010': 66,
         '1110': 63,
         '1001': 63,
         '1111': 63,
         '0111': 57,
         '0001': 56,
         '0110': 54,
         '0010': 52,
         '0100': 52,
         '0000': 52})

In [65]:
# hidden tests in this cell will be used for grading.